In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os, time


def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


X = tf.placeholder(tf.float32, shape=[None, 784])

D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))

D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))

theta_D = [D_W1, D_W2, D_b1, D_b2]


Z = tf.placeholder(tf.float32, shape=[None, 100])

G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))

G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))

theta_G = [G_W1, G_W2, G_b1, G_b2]


def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def generator(z):
    G_h1 = tf.nn.relu(tf.matmul(z, G_W1) + G_b1)
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)

    return G_prob


def discriminator(x):
    D_h1 = tf.nn.relu(tf.matmul(x, D_W1) + D_b1)
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)

    return D_prob, D_logit


def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig


G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1. - D_fake))
# G_loss = -tf.reduce_mean(tf.log(D_fake))

# Alternative losses:
# -------------------
D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, 
                                                                     labels=tf.ones_like(D_logit_real)))
D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, 
                                                                     labels=tf.zeros_like(D_logit_fake)))

D_loss = D_loss_real + D_loss_fake
G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, 
                                                                labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=theta_D)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=theta_G)

mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)

if not os.path.exists('out/'):
    os.makedirs('out/')

Extracting ../../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data/t10k-labels-idx1-ubyte.gz


In [34]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

t0 = time.time()
mb_size = 128
Z_dim = 100
i = 0
for it in range(1000000):
    #     if it % 1000 == 0:
    #         samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})

    #         fig = plot(samples)
    #         plt.savefig('out/{}.png'.format(str(i).zfill(3)), bbox_inches='tight')
    #         i += 1
    #         plt.close(fig)

    X_mb, _ = mnist.train.next_batch(mb_size)

    _, d_loss, d_real, d_fake = sess.run([D_solver, D_loss, D_real, D_fake], 
                                              feed_dict={X: X_mb, Z: sample_Z(mb_size, Z_dim)})
    _, g_loss, g_fake = sess.run([G_solver, G_loss, D_fake], 
                                      feed_dict={Z: sample_Z(mb_size, Z_dim)})

    d_acc = 0.5 * (np.where(d_fake[:, 0] > 0.5, 0, 1).sum(0) + np.where(d_real[:, 0] > 0.5, 1, 0).sum(0)) / mb_size
    g_acc = np.where(g_fake[:, 0] > 0.5, 1, 0).sum(0) / mb_size
    
    if it % 1000 == 0:
        print('{:6d} -- G: {:.5f}, {:.5f};  D: {:.5f}, {:.5f};  [{:.1f}s]'\
              .format(it, g_loss, g_acc, d_loss, d_acc, time.time() - t0))
        t0 = time.time()

     0 -- G: 2.46270, 0.00000;  D: 1.23104, 0.52344;  [0.1s]
  1000 -- G: 9.63404, 0.00000;  D: 0.01260, 0.99609;  [5.5s]
  2000 -- G: 5.79360, 0.00000;  D: 0.02305, 1.00000;  [5.5s]
  3000 -- G: 5.09898, 0.00000;  D: 0.06944, 0.99219;  [5.5s]
  4000 -- G: 4.80478, 0.00000;  D: 0.20574, 0.96094;  [5.6s]
  5000 -- G: 4.69415, 0.00781;  D: 0.14558, 0.98047;  [5.5s]
  6000 -- G: 4.15293, 0.03906;  D: 0.37866, 0.91797;  [5.6s]
  7000 -- G: 2.76031, 0.10156;  D: 0.70560, 0.86719;  [5.6s]
  8000 -- G: 3.69904, 0.05469;  D: 0.71437, 0.85156;  [5.5s]
  9000 -- G: 2.55201, 0.15625;  D: 0.64850, 0.86328;  [5.5s]
 10000 -- G: 3.34503, 0.04688;  D: 0.52646, 0.88672;  [5.6s]
 11000 -- G: 3.07995, 0.07031;  D: 0.56708, 0.89062;  [5.5s]
 12000 -- G: 2.61995, 0.10938;  D: 0.70089, 0.84375;  [5.5s]
 13000 -- G: 3.11949, 0.03906;  D: 0.57957, 0.87500;  [5.6s]
 14000 -- G: 2.69086, 0.10156;  D: 0.79068, 0.85547;  [5.5s]
 15000 -- G: 2.62260, 0.08594;  D: 0.56110, 0.85547;  [5.5s]
 16000 -- G: 2.28213, 0.

KeyboardInterrupt: 